In [1]:
# this notebook should stay in root
"""
Download proteins and other features from InterPro for a given pfam protein family
"""
import sys
sys.path.append('..')
import json
import sys
import random
import requests
from urllib.error import HTTPError
from urllib.request import urlopen
from time import sleep
from requests.adapters import HTTPAdapter, Retry
import re

In [6]:
re_next_link = re.compile(r'<(.+)>; rel="next"')
retries = Retry(total=5, backoff_factor=0.25, status_forcelist=[500, 502, 503, 504])
session = requests.Session()
session.mount("https://", HTTPAdapter(max_retries=retries))
results = []

def get_next_link(headers):
    if "Link" in headers:
        match = re_next_link.match(headers["Link"])
        if match:
            return match.group(1)

def get_batch(batch_url):
    while batch_url:
        response = session.get(batch_url)
        response.raise_for_status()
        total = response.headers["x-total-results"]
        yield response, total
        batch_url = get_next_link(response.headers)

url = 'https://rest.uniprot.org/uniprotkb/search?format=json&query=%28*%29&size=500'
interactions = {}
numbers_done = []
while len(results) <= 50000:
    taxonomy_number = random.randint(1, 300000)
    if taxonomy_number in numbers_done:
        continue
    numbers_done.append(taxonomy_number)
    url = f'https://rest.uniprot.org/uniprotkb/search?format=json&query=organism_id:{taxonomy_number}&size=500'
    for batch, total in get_batch(url):
        json_object = json.loads(batch.text)
        tmp_results = json_object['results']
        # print(json_object['results'][0].keys())
        for i in tmp_results:
            results.append(i)
        print(f'{len(results)} / {total}')
        tmp_batch += len(results)
        break

0 / 0
2 / 2
2 / 0
2 / 0
2 / 0
2 / 0
2 / 0
2 / 0
2 / 0
251 / 249
251 / 0
251 / 0
251 / 0
263 / 12
266 / 3
291 / 25
291 / 0
293 / 2
293 / 0
321 / 28
321 / 0
322 / 1
322 / 0
322 / 0
322 / 0
322 / 0
322 / 0
326 / 4
326 / 0
326 / 0
326 / 0
326 / 0
326 / 0
326 / 0
326 / 0
326 / 0
327 / 1
383 / 56
383 / 0
383 / 0
386 / 3
387 / 1
389 / 2
389 / 0
393 / 4
393 / 0
393 / 0
393 / 0
393 / 0
611 / 218
613 / 2
613 / 0
613 / 0
614 / 1
618 / 4
618 / 0
618 / 0
702 / 84
702 / 0
702 / 0
709 / 7
724 / 15
724 / 0
724 / 0
724 / 0
724 / 0
733 / 9
733 / 0
737 / 4
737 / 0
737 / 0
737 / 0
737 / 0
737 / 0
976 / 239
977 / 1
1477 / 9290
1477 / 0
1477 / 0
1478 / 1
1823 / 345
1826 / 3
1831 / 5
1831 / 0
1834 / 3
2334 / 888
2335 / 1
2345 / 10
2345 / 0
2345 / 0
2345 / 0
2345 / 0
2345 / 0
2345 / 0
2345 / 0
2345 / 0
2345 / 0
2345 / 0
2345 / 0
2345 / 0
2345 / 0
2345 / 0
2345 / 0
2356 / 11
2356 / 0
2356 / 0
2356 / 0
2356 / 0
2356 / 0
2356 / 0
2356 / 0
2356 / 0
2356 / 0
2359 / 3
2361 / 2
2418 / 57
2418 / 0
2418 / 0
2418 / 0
2

In [11]:
len(results)

50167

In [12]:
import os
import pickle

# Create a directory named "data" in the current working directory if it doesn't exist
data_dir = "data_random_selection_uniParc"
os.makedirs(data_dir, exist_ok=True)

# Save filtered_data as a .p file
filtered_data_file = os.path.join(data_dir, "not_filtered_data_uniParc.p")
with open(filtered_data_file, "wb") as file:
    pickle.dump(results, file)

In [9]:
# OPEN UNFILTERED DATA------------------------------------

In [10]:
len(results)

50167

In [13]:
# Process the downloaded data TODO------------------------------------
total_len = 0
total_sequences = 0
filtered_data = []

count = 1
for sequence in results:
    if sequence:
        keys = sequence.keys()
        if 'sequence' in keys:
            seq = sequence['sequence']
            # print(seq)
            length = seq["length"]
            total_len += length
            total_sequences += 1
            filtered_data.append(sequence)
            # for debugging:
            # break
    
    
print("Sequences retrieved: ", total_sequences)
print("Sequence average length: ", total_len/total_sequences)

Sequences retrieved:  50167
Sequence average length:  392.13287619351365


In [14]:
import random

# Calculate the number of entries to select (10% of the data)
# ID_test_percentage = 0.1
# num_entries_to_select = int(len(filtered_data) * ID_test_percentage)

# Randomly select ID_test_percentage of the data entries
# random_selection = random.sample(filtered_data, num_entries_to_select)

# Print the randomly selected entries
#for entry in random_selection:
#    print(entry)
#    break

# Remove the randomly selected entries from `filtered_data` if needed
# filtered_data = [entry for entry in filtered_data if entry not in random_selection]
print("Sequences retrieved: ", total_sequences)
print("Sequence average length: ", total_len/total_sequences)

Sequences retrieved:  50167
Sequence average length:  392.13287619351365


In [15]:
print(len(filtered_data))

50167


In [16]:
import os
import pickle

# Create a directory named "data" in the current working directory if it doesn't exist
data_dir = "data_random_selection_uniParc"
os.makedirs(data_dir, exist_ok=True)

# Save filtered_data as a .p file
filtered_data_file = os.path.join(data_dir, "filtered_data.p")
with open(filtered_data_file, "wb") as file:
    pickle.dump(filtered_data, file)

In [17]:
# Save random_selection of 10 proteins as a .p file
random_selection = random.sample(filtered_data, 10)
random_selection_file = os.path.join(data_dir, "random_selection_10.p")
with open(random_selection_file, "wb") as file:
    pickle.dump(random_selection, file)
# Save random_selection of 100 proteins as a .p file
random_selection = random.sample(filtered_data, 100)
random_selection_file = os.path.join(data_dir, "random_selection_100.p")
with open(random_selection_file, "wb") as file:
    pickle.dump(random_selection, file)
# Save random_selection of 500 proteins as a .p file
random_selection = random.sample(filtered_data, 500)
random_selection_file = os.path.join(data_dir, "random_selection_500.p")
with open(random_selection_file, "wb") as file:
    pickle.dump(random_selection, file)
# Save random_selection of 1000 proteins as a .p file
random_selection = random.sample(filtered_data, 1000)
random_selection_file = os.path.join(data_dir, "random_selection_1000.p")
with open(random_selection_file, "wb") as file:
    pickle.dump(random_selection, file)
# Save random_selection of 2000 proteins as a .p file
random_selection = random.sample(filtered_data, 2000)
random_selection_file = os.path.join(data_dir, "random_selection_2000.p")
with open(random_selection_file, "wb") as file:
    pickle.dump(random_selection, file)
# Save random_selection of 5000 proteins as a .p file
random_selection = random.sample(filtered_data, 5000)
random_selection_file = os.path.join(data_dir, "random_selection_5000.p")
with open(random_selection_file, "wb") as file:
    pickle.dump(random_selection, file)
# Save random_selection of 10000 proteins as a .p file
random_selection = random.sample(filtered_data, 10000)
random_selection_file = os.path.join(data_dir, "random_selection_10000.p")
with open(random_selection_file, "wb") as file:
    pickle.dump(random_selection, file)